In [60]:
import pandas as pd 
import numpy as np 
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn.compose import TransformedTargetRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import QuantileTransformer
from sklearn import metrics
from sklearn.metrics import mean_squared_error, r2_score

df= pd.read_csv("filtered_062019.csv",index_col=0)
df1=df.dropna()
df1 = df1.drop_duplicates()
df1= df1.drop(['gpu0.mem_max', 'gpu0.smUtil_avg', 'gpu0.smUtil_max', 'gpu1.mem_max', 'gpu1.smUtil_avg', 'gpu1.smUtil_max', 'gpu2.mem_max', 'gpu2.smUtil_avg', 'gpu2.smUtil_max', 'gpu3.mem_max', 'gpu3.smUtil_avg', 'gpu3.smUtil_max'],axis=1)
df1=df1.drop(['run_count', 'start', 'status', 'wait_time','cluster', 'ctime', 'datetime', 'end', 'etime', 'exec_host', 'exec_vnode', 'group', 'job_id', 'qtime'],axis=1)
df1=df1.drop(['Exit_status', 'Resource_List.fluent_lic','Resource_List.mpiprocs','Resource_List.ngpus','resources_used.GPU_duration','resources_used.GPU_maxGpuMemoryUsed', 'resources_used.GPU_energyConsumed','Resource_List.nodect'],axis=1)
df1['CPU_eff']= (df1['resources_used.cput']/df1['resources_used.walltime'])/df1['Resource_List.ncpus']
df1 = df1.replace([np.inf, -np.inf], np.nan)
df1=df1.dropna()
X = df1.drop('CPU_eff', axis=1)
y = df1['CPU_eff']


X_encoded = pd.get_dummies(X, prefix_sep='_', drop_first=True)

X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2,random_state=0)

#lab_enc = preprocessing.LabelEncoder()
#y_train_encoded = lab_enc.fit_transform(y_train)
#y_test_encoded = lab_enc.fit_transform(y_test)
X_train_scaled= StandardScaler().fit_transform(X_train)
X_test_scaled= StandardScaler().fit_transform(X_test)


transformer = QuantileTransformer(output_distribution='normal')
regressor = LinearRegression()
regr = TransformedTargetRegressor(regressor=regressor,
                                   transformer=transformer)
regr.fit(X_train_scaled, y_train)

#print('R2 score: {0:.2f}'.format(regr.score(X_test_scaled, y_test)))

raw_target_regr = LinearRegression().fit(X_train_scaled, y_train)
#print('R2 score: {0:.2f}'.format(raw_target_regr.score(X_test_scaled, y_test)))

#Believe the R-Squared term is bad because features have not been scaled properly. 

#Predictions using test data, where 1 is perfect prediction
cpu_predictor_y= regr.predict(X_test_scaled)
print('Coefficient of determination: %.2f'
      % r2_score(y_test_encoded, cpu_predictor_y))
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test_encoded, cpu_predictor_y))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test_encoded, cpu_predictor_y))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test_encoded, cpu_predictor_y)))

#Explanation of Pipeline: 
#Data was loaded in pd.read, columns with NaN were dropped immediately. As after doing EDA, the column with NaN was deemed irrelevant.
#Next, columns with GPU related functions were also dropped because they have no physical relation with CPU usage based on intuition.
#Thereafter, columns with zero variance were dropped, followed by redundant physical interpretations like admin status or date-time.
#New column CPU_eff was inserted based on the formula given.
#All values that contained inf were replaced with NaN and dropped. 

#With the dataset, X -> contained 12 features in total and y -> was the target variable, CPU_eff

#Categorical features in X were user,dept,queue and were converted to numerical values using get_dummies from pandas 

#X and y were split into train and test dataset, before using label encoder to fit the target variable y. Standardscaler was then used to fit and transform X

#X and y were then fed into LinearRegression() to generate a multivariable regression model and the evaluation metrics are shown below.

#Analysis of coefficient of determination: This negative R-squared term implies that perhaps the model confused several instances with itself, and confused
#the first instance with the fifth instance for example. Or perhaps the features used were not strong enough to form any predictive capability 
#Or maybe the data was not scaled properly according to the target variable. 

Coefficient of determination: -1.54
